# [모듈 7.1] 리소스 정리

참조 : 
- [Boto3 API 가이드](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/frauddetector.html#FraudDetector.Client.delete_label)

# 0. 환경 셋업

In [37]:
# detectorId
%store -r DETECTOR_NAME 
# eventTypeName
%store -r EVENT_TYPE
%store -r ENTITY_TYPE
%store -r eventVariables
%store -r rule_list
%store -r detector_version

%store -r MODEL_NAME
%store -r modelVersionNumber
%store -r outcomes

In [35]:
import boto3
client = boto3.client("frauddetector")

from IPython.display import display as dp

# 1. Detector 관련 리소스 제거

## Detector Version 을 Active 로 변경
- 아래 업데이트는 Status 를 Draft --> Active 로 변환 함.

In [16]:
response = client.update_detector_version_status(
    detectorId= DETECTOR_NAME,
    detectorVersionId= detector_version,
    status='INACTIVE'
)
print("\n -- detector inactivated -- ")
print(response)


 -- detector inactivated -- 
{'ResponseMetadata': {'RequestId': '6cf9ba12-4e83-4f97-9518-4e02ba008211', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Sun, 12 Sep 2021 23:31:06 GMT', 'content-type': 'application/x-amz-json-1.1', 'content-length': '2', 'connection': 'keep-alive', 'x-amzn-requestid': '6cf9ba12-4e83-4f97-9518-4e02ba008211'}, 'RetryAttempts': 0}}


## Detector Version 제거

In [17]:
response = client.delete_detector_version(
    detectorId= DETECTOR_NAME,
    detectorVersionId = detector_version
)

In [18]:
response

{'ResponseMetadata': {'RequestId': '7d9a930b-b500-48da-93f4-c7eea26c2d76',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Sun, 12 Sep 2021 23:31:07 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amzn-requestid': '7d9a930b-b500-48da-93f4-c7eea26c2d76'},
  'RetryAttempts': 0}}

## Rule 제거

In [19]:
rule_list

[{'ruleId': 'rule0_adtaking_fraud_phase0_model_20210912',
  'ruleVersion': '1',
  'detectorId': 'adtaking_fraud_phase0_detector_20210912'},
 {'ruleId': 'rule1_adtaking_fraud_phase0_model_20210912',
  'ruleVersion': '1',
  'detectorId': 'adtaking_fraud_phase0_detector_20210912'},
 {'ruleId': 'rule2_adtaking_fraud_phase0_model_20210912',
  'ruleVersion': '1',
  'detectorId': 'adtaking_fraud_phase0_detector_20210912'},
 {'ruleId': 'rule3_adtaking_fraud_phase0_model_20210912',
  'ruleVersion': '1',
  'detectorId': 'adtaking_fraud_phase0_detector_20210912'}]

In [20]:
for rule in rule_list:
    response = client.delete_rule(
        rule= rule                
    )
    print(response)


{'ResponseMetadata': {'RequestId': 'e7a939e8-aba9-4969-9371-faa5ec7d4043', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Sun, 12 Sep 2021 23:31:10 GMT', 'content-type': 'application/x-amz-json-1.1', 'content-length': '2', 'connection': 'keep-alive', 'x-amzn-requestid': 'e7a939e8-aba9-4969-9371-faa5ec7d4043'}, 'RetryAttempts': 0}}
{'ResponseMetadata': {'RequestId': 'f8bd5703-88ab-4d5d-ab10-71bd1c077d93', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Sun, 12 Sep 2021 23:31:11 GMT', 'content-type': 'application/x-amz-json-1.1', 'content-length': '2', 'connection': 'keep-alive', 'x-amzn-requestid': 'f8bd5703-88ab-4d5d-ab10-71bd1c077d93'}, 'RetryAttempts': 0}}
{'ResponseMetadata': {'RequestId': 'c3f28f32-31c1-4d8f-a506-367d924ee11e', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Sun, 12 Sep 2021 23:31:11 GMT', 'content-type': 'application/x-amz-json-1.1', 'content-length': '2', 'connection': 'keep-alive', 'x-amzn-requestid': 'c3f28f32-31c1-4d8f-a506-367d924ee11e'}, 'RetryAttempts': 0}

## Detector 제거

In [21]:
response = client.delete_detector(
    detectorId= DETECTOR_NAME
)
response

{'ResponseMetadata': {'RequestId': '311a9022-9a6f-4d15-ad10-ac3bfcaa21d2',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Sun, 12 Sep 2021 23:31:13 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amzn-requestid': '311a9022-9a6f-4d15-ad10-ac3bfcaa21d2'},
  'RetryAttempts': 0}}

# 2. Model 제거

## Model Status 변경
- `ACTIVE` -->  `INACTIVE` 로 변경

In [22]:
response = client.update_model_version_status (
    modelId = MODEL_NAME,
    modelType = 'ONLINE_FRAUD_INSIGHTS',
    modelVersionNumber = modelVersionNumber,
    status = 'INACTIVE'
)
print("-- inactivating model --")
print(response)



-- activating model --
{'ResponseMetadata': {'RequestId': '8e7708fb-39e8-4520-99cf-f05c27f22ea9', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Sun, 12 Sep 2021 23:31:36 GMT', 'content-type': 'application/x-amz-json-1.1', 'content-length': '2', 'connection': 'keep-alive', 'x-amzn-requestid': '8e7708fb-39e8-4520-99cf-f05c27f22ea9'}, 'RetryAttempts': 0}}


In [26]:
import time
#-- wait until model is active 
print("--- waiting until model status is inactive ")
stime = time.time()
while True:
    response = client.get_model_version(modelId=MODEL_NAME, 
                                        modelType = "ONLINE_FRAUD_INSIGHTS", modelVersionNumber = '1.0')
    if response['status'] != 'TRAINING_COMPLETE':
        print(f"current progress: {(time.time() - stime)/60:{3}.{3}} minutes")
        time.sleep(30)  # sleep for #0s
    if response['status'] == 'TRAINING_COMPLETE':
        print("Model status : " +  response['status'])
        break
        
etime = time.time()
print("Elapsed time : %s" % (etime - stime) + " seconds \n"  )
print(response)

--- waiting until model status is inactive 
Model status : TRAINING_COMPLETE
Elapsed time : 0.22902369499206543 seconds 

{'modelId': 'adtaking_fraud_phase0_model_20210912', 'modelType': 'ONLINE_FRAUD_INSIGHTS', 'modelVersionNumber': '1.0', 'trainingDataSource': 'EXTERNAL_EVENTS', 'trainingDataSchema': {'modelVariables': ['str_ip', 'str_app', 'str_device', 'str_os', 'str_channel'], 'labelSchema': {'labelMapper': {'FRAUD': ['1'], 'LEGIT': ['0']}}}, 'externalEventsDetail': {'dataLocation': 's3://sagemaker-us-east-1-057716757052/adtalking_fraud_phase0/train/train-180000.csv', 'dataAccessRoleArn': 'arn:aws:iam::057716757052:role/AFD-gsmoon'}, 'status': 'TRAINING_COMPLETE', 'arn': 'arn:aws:frauddetector:us-east-1:057716757052:model-version/ONLINE_FRAUD_INSIGHTS/adtaking_fraud_phase0_model_20210912/1.0', 'ResponseMetadata': {'RequestId': '37799abb-10cb-4f7b-94a8-7d5235eaf1b9', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Mon, 13 Sep 2021 01:03:21 GMT', 'content-type': 'application/x-amz-j

## 모델 버전 제거

In [27]:
response = client.delete_model_version(
    modelId= MODEL_NAME, 
    modelType='ONLINE_FRAUD_INSIGHTS',
    modelVersionNumber= modelVersionNumber,
)
response

{'ResponseMetadata': {'RequestId': 'd4c87620-3abe-404d-945e-62c10856c4da',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Mon, 13 Sep 2021 01:13:47 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'd4c87620-3abe-404d-945e-62c10856c4da'},
  'RetryAttempts': 0}}

## 모델 제거

In [28]:
response = client.delete_model(
    modelId= MODEL_NAME, 
    modelType='ONLINE_FRAUD_INSIGHTS'
)
response

{'ResponseMetadata': {'RequestId': '9d4f0ae0-3a77-4dd7-ae78-dc4236a7cf52',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Mon, 13 Sep 2021 01:15:26 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amzn-requestid': '9d4f0ae0-3a77-4dd7-ae78-dc4236a7cf52'},
  'RetryAttempts': 0}}

# 3. 이벤트 타입 제거

In [29]:
response = client.delete_event_type(
    name= EVENT_TYPE,
)
response

{'ResponseMetadata': {'RequestId': '45f0af60-9005-459e-b74b-1e85a275c010',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Mon, 13 Sep 2021 01:21:09 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amzn-requestid': '45f0af60-9005-459e-b74b-1e85a275c010'},
  'RetryAttempts': 0}}

## 4.엔터티 타입 제거

In [32]:
response = client.delete_entity_type(
    name = ENTITY_TYPE,
)
response

{'ResponseMetadata': {'RequestId': '640e683b-6f1a-484b-87d1-473d3ef09995',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Mon, 13 Sep 2021 01:24:08 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amzn-requestid': '640e683b-6f1a-484b-87d1-473d3ef09995'},
  'RetryAttempts': 0}}

## 5. Outcome 제거

In [36]:
for outcome in outcomes:
    print(outcome)
    response = client.delete_outcome(
        name= outcome
    )    
    dp(response)

fraud


ConflictException: An error occurred (ConflictException) when calling the DeleteOutcome operation: You cannot remove this outcome because it is currently in use by at least one rule.

## 6. Variables 제거

In [39]:
for var in eventVariables:
    print(var)
    response = client.delete_variable(
        name='string'
    )    
    dp(response)

str_ip


{'ResponseMetadata': {'RequestId': '9059f132-b687-47de-a54c-f33e4078bd02',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Mon, 13 Sep 2021 01:35:37 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amzn-requestid': '9059f132-b687-47de-a54c-f33e4078bd02'},
  'RetryAttempts': 0}}

str_app


{'ResponseMetadata': {'RequestId': 'f4e3b6c7-6e78-4a45-9265-ac7b1c4a03c5',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Mon, 13 Sep 2021 01:35:38 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'f4e3b6c7-6e78-4a45-9265-ac7b1c4a03c5'},
  'RetryAttempts': 0}}

str_device


{'ResponseMetadata': {'RequestId': 'd4e8b339-9fee-4845-87c5-bc5eb98fe659',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Mon, 13 Sep 2021 01:35:38 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'd4e8b339-9fee-4845-87c5-bc5eb98fe659'},
  'RetryAttempts': 0}}

str_os


{'ResponseMetadata': {'RequestId': 'f5a9a9c8-1b70-40aa-9b54-ee443f106c53',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Mon, 13 Sep 2021 01:35:38 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'f5a9a9c8-1b70-40aa-9b54-ee443f106c53'},
  'RetryAttempts': 0}}

str_channel


{'ResponseMetadata': {'RequestId': '54975787-effc-4794-8bbc-fbbd805ee057',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Mon, 13 Sep 2021 01:35:38 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amzn-requestid': '54975787-effc-4794-8bbc-fbbd805ee057'},
  'RetryAttempts': 0}}

# 7. 레이블 제거
- UI 참조

In [ ]:
# response = client.delete_label(
#     name='string'
# )